# Tokenize Markdown

In [ ]:
%run ../functions.ipynb
%run ./functions.ipynb

In [ ]:
# create schema
execute_query("""
--DROP TABLE IF EXISTS mkd01;
CREATE TABLE IF NOT EXISTS mkd01 (
    hash TEXT PRIMARY KEY,
    reference TEXT,
    index NUMERIC,
    content TEXT,
    metadata JSONB,
    embedding vector(1500)
);
""")

In [11]:
print("start")
for file in find_sources('./docs-onefront'):
    print(file)
    index_markdown(file)
print("done")

start
./docs-onefront/index.md
./docs-onefront/changelog/index.md
./docs-onefront/environment-setup/index.md
./docs-onefront/proxy-service/index.md
./docs-onefront/proxy-service/logs/index.md
./docs-onefront/registry-service/index.md
./docs-onefront/registry-service/jwt/index.md
./docs-onefront/registry-service/shared-source/index.md
./docs-onefront/registry-service/upstream-downstream/index.md
./docs-onefront/development-environment/index.md
./docs-onefront/development-environment/how-to-install-docker-cli/index.md
./docs-onefront/development-environment/how-to-install-podman/index.md
./docs-onefront/development-environment/snyk-integration/how-snyk-works/index.md
./docs-onefront/development-environment/snyk-integration/snyk-setup/index.md
./docs-onefront/development-environment/snyk-integration/what-is-snyk/index.md
./docs-onefront/development-environment/environment-and-os/index.md
./docs-onefront/development-environment/how-to-install-wsl/index.md
./docs-onefront/development-enviro

In [14]:
print("start")
for file in find_sources('./docs-vapor'):
    print(file)
    index_markdown(file)
print("done")

start
./docs-vapor/BEST_PRACTICE.md
./docs-vapor/NEXUS_TOKEN.md
./docs-vapor/README.md
./docs-vapor/CONTRIB.md
embed: ./docs-vapor/CONTRIB.md
./docs-vapor/CONTRIBUTING.md
./docs-vapor/DOCKER_NEXUS_TOKEN.md
./docs-vapor/how-to-use-renovate/README.md
./docs-vapor/how-to-create-a-contrib-component/README.md
./docs-vapor/how-to-report-an-issue-and-fix-a-bug/README.md
./docs-vapor/how-to-add-new-icon/README.md
./docs-vapor/how-to-ensure-code-consistency-between-windows-and-linux-machines/README.md
./docs-vapor/gitflow/README.md
./docs-vapor/cicd/README.md
./docs-vapor/how-to-create-a-contrib-project/README.md
./docs-vapor/how-to-create-and-download-build-storybook/README.md
./docs-vapor/how-to-work-in-gitpod/README.md
done


In [6]:
def search(query, limit = 1):
    q1 = generate_embedding(query, dimensions=1500)
    results = execute_query("""
        SELECT
            CASE
                WHEN reference LIKE './docs-vapor%%'
                THEN 'https://development.teamsystem.com/docs/default/Component/vapor-react'
                ELSE 'https://development.teamsystem.com/docs/default/System/one-front'
            END as base_url,
            regexp_replace(regexp_replace(regexp_replace(reference, E'^\\./[^/]+', ''), E'/(index|readme)\\.md$', '', 'i'), E'\\.md$', '', 'i') as path,
            regexp_replace(REPLACE(LOWER(REGEXP_REPLACE(COALESCE(metadata->>'Header 2', metadata->>'Header 1'), E'[^a-zA-Z0-9 ]', '', 'g')), ' ', '-'), E'-+', '-', 'g') as fragment
        FROM mkd01 ORDER BY embedding <-> %s::vector LIMIT %s
""", [q1, limit]);

    # Process each row to format it correctly
    formatted_urls = []
    for row in results:
        base_url, path, fragment = row
        # Ensure there is a '/' between base_url and path if not present
        if not base_url.endswith('/') and not path.startswith('/'):
            path = '/' + path
        full_url = f"{base_url}{path}#{fragment}"
        formatted_urls.append(full_url)
    
    return formatted_urls

In [7]:
print_rows(search("i want to create a new component", 1));

https://development.teamsystem.com/docs/default/Component/vapor-react/BEST_PRACTICE#there-are-some-ways-to-create-a-new-component


In [ ]:
print_rows(search("i need fontawesome token", 1));

In [9]:
print_rows(search("come si mocca una chiave del registry?", 1));

https://development.teamsystem.com/docs/default/System/one-front/tutorials/proxy-service/mock-buckets#read-the-mocked-value


In [ ]:
print_rows(search("Dua të krijoj një komponent të ri", 1));

In [8]:
print_rows(search("Dua te ndertoj nje buton nga vapor", 1));

https://development.teamsystem.com/docs/default/Component/vapor-react/#how-to-use-vapor


In [15]:
print_rows(search("I want to authenticate my docker", 1));

https://development.teamsystem.com/docs/default/System/one-front/environment-setup#docker-login


In [16]:
print_rows(search("How do i setup wsl and vscode on windows?", 1));

https://development.teamsystem.com/docs/default/System/one-front/development-environment/how-to-install-wsl#step-3-install-vscode-remotewsl
